<a href="https://colab.research.google.com/github/danielbauer1860/LDS_Project/blob/main/generation/generating_fic_4epoch_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook generates the output for the model that was fine-tuned on the fictional dataset for 4 epochs. For further details regarding the implementation refer to `generating_aca_2epoch_dataset.ipynb`, as that notebook is documented the most extensively.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


This notebook references [the project of Salminen et al. (2021)](https://github.com/joolsa/FakeReviews/blob/)

In [ ]:
!pip install iteround -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 tensorboard huggingface_hub[cli] xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 101.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Loading the model

Redownloading the model required some tweaks, which are described in:
https://stackoverflow.com/questions/76459034/how-to-load-a-fine-tuned-peft-lora-model-based-on-llama-with-huggingface-transfo

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from accelerate import infer_auto_device_map, init_empty_weights

peft_model_id = 'dbauer1860/llama-2-bnc-baby-fictional-4-epochs'

config = PeftConfig.from_pretrained(peft_model_id)

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    torch_dtype='auto',
    device_map='auto',
    offload_folder="offload", offload_state_dict = True
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/134M [00:00<?, ?B/s]

# Extracting Prompts

In [ ]:
import pandas as pd
import numpy as np
import random
import nltk
from iteround import saferound
from nltk.tokenize import word_tokenize

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Linguistic Data Science/data/bnc_baby_texts.csv', sep='|')

In [ ]:
df.head()

,text,category
0,"Why do certain images matter to one, and why i...",ACA
1,"In many developing countries, the immunogenici...",ACA
2,Anthony Kenny The British Library 1992 Scholar...,ACA
3,Beforehand he worked as a probation officer fo...,ACA
4,CONTINUITY AND CHANGE are features of every po...,ACA


In [ ]:
def split_into_sentences(text):
    return nltk.sent_tokenize(text)

In [ ]:
df['sentences'] = df['text'].apply(split_into_sentences)

In [ ]:
df

,text,category,sentences
0,"Why do certain images matter to one, and why i...",ACA,"[Why do certain images matter to one, and why ..."
1,"In many developing countries, the immunogenici...",ACA,"[In many developing countries, the immunogenic..."
2,Anthony Kenny The British Library 1992 Scholar...,ACA,[Anthony Kenny The British Library 1992 Schola...
3,Beforehand he worked as a probation officer fo...,ACA,[Beforehand he worked as a probation officer f...
4,CONTINUITY AND CHANGE are features of every po...,ACA,[CONTINUITY AND CHANGE are features of every p...
...,...,...,...
177,"Artworks THE Friends of Oriel Ynys Mon, Llange...",NEWS,"[Artworks THE Friends of Oriel Ynys Mon, Llang..."
178,THERE will be those who portray the report of ...,NEWS,[THERE will be those who portray the report of...
179,BUCKINGHAM PALACE 30 September: The Princess R...,NEWS,[BUCKINGHAM PALACE 30 September: The Princess ...
180,The Scottish Landowners' Federation unequivoca...,NEWS,[The Scottish Landowners' Federation unequivoc...


In [ ]:
random_state = 14
random_sentence_state = 1860
sample_count = 20

In [ ]:
def get_random_sentence(sentences_list, random_state=random_sentence_state, min_word_count=5):
    random.seed(random_state)
    filtered_sentences = [sentence for sentence in sentences_list if len(word_tokenize(sentence)) > min_word_count]
    return random.choice(filtered_sentences) if filtered_sentences else None

In [ ]:
df['prompt'] = df['sentences'].apply(get_random_sentence)

In [ ]:
def get_samples(df, n=sample_count, state=random_state):
  return df.sample(n=n, random_state=state)

In [ ]:
df_aca, df_dem, df_fic, df_news = [get_samples(y) for x, y in df.groupby(['category'])]

<ipython-input-16-c993aa370679>:1: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  df_aca, df_dem, df_fic, df_news = [get_samples(y) for x, y in df.groupby(['category'])]


In [ ]:
df_news

,text,category,sentences,prompt
129,THE couch cavorted upon by Joanne Whalley-Kilm...,NEWS,[THE couch cavorted upon by Joanne Whalley-Kil...,"They begged Rytasha for reimbursement, but non..."
179,BUCKINGHAM PALACE 30 September: The Princess R...,NEWS,[BUCKINGHAM PALACE 30 September: The Princess ...,FISA was then a relatively unimportant body wh...
123,"West led the queen of clubs, and declarer won ...",NEWS,"[West led the queen of clubs, and declarer won...",The reason behind this concentrated specialist...
138,PRIZES will be awarded for the best carnations...,NEWS,[PRIZES will be awarded for the best carnation...,The revolution in glasshouses had surely start...
161,The disaster was all the more stunning because...,NEWS,[The disaster was all the more stunning becaus...,‘I must stress that the involvement of armed f...
172,Sunbed scare… Could this machine damage your h...,NEWS,[Sunbed scare… Could this machine damage your ...,More than 500 staff and 450 GPs who are believ...
175,"MORE than 5,000 workers have swamped Hamilton ...",NEWS,"[MORE than 5,000 workers have swamped Hamilton...",The discounts include 3pc off the standard loa...
124,Morning sickness good for mothers By Roger Hig...,NEWS,[Morning sickness good for mothers By Roger Hi...,These organisations include the Woodland Trust...
109,"IF THIS WAS HAPPINESS by Barbara Leaming, Weid...",NEWS,"[IF THIS WAS HAPPINESS by Barbara Leaming, Wei...","Realising that she had not loved him, Boris th..."
87,"Abandoned fawns have been reared on a bottle, ...",NEWS,"[Abandoned fawns have been reared on a bottle,...",Treatment can cost anything from a few hundred...


In [ ]:
aca_prompts = df_aca['prompt'].to_list()
fic_prompts = df_fic['prompt'].to_list()
news_prompts = df_news['prompt'].to_list()

# Generating output

In [ ]:
#Generation parameters; these were, for the most part, set in accordance to Salminen et al.
do_sample = True
top_k = 0
top_p = 0.92
temperature = 0.7
eos_token = model.config.eos_token_id
device = "cuda:0"
max_length = 1000
repetition_penalty = 1.1

In [ ]:
#from Salminen et al.; a seed value was set to allow the generation to be re-produced and continued over multiple sessions
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    random.seed(seed_value) # Python
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False

The generating loop

In [ ]:
def generate(prompts):
  outputs = []
  for prompt in prompts:
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    random_seed(seed_value=random_state, use_cuda=True)
    o = model.generate(**inputs, do_sample=do_sample, top_k=top_k, top_p=top_p, temperature=temperature, eos_token_id=eos_token, max_length=max_length, repetition_penalty=repetition_penalty)
    outputs.append(tokenizer.decode(o[0], skip_special_tokens=True))
  return outputs

In [ ]:
aca_outputs = generate(aca_prompts)

In [ ]:
fic_outputs = generate(fic_prompts)

In [ ]:
news_outputs = generate(news_prompts)

In [ ]:
aca_results = pd.DataFrame(aca_outputs, columns=['output'])
aca_results['prompt_category'] = 'ACA'
aca_results['prompt'] = aca_prompts

fic_results = pd.DataFrame(fic_outputs, columns=['output'])
fic_results['prompt_category'] = 'FIC'
fic_results['prompt'] = fic_prompts

news_results = pd.DataFrame(news_outputs, columns=['output'])
news_results['prompt_category'] = 'NEWS'
news_results['prompt'] = news_prompts

result_frames = [aca_results, fic_results, news_results]

result_df = pd.concat(result_frames, ignore_index=True)
result_df

,output,prompt_category,prompt
0,Genesis: An Authorship Study in Computer-assis...,ACA,Genesis: An Authorship Study in Computer-assis...
1,It should be obvious why a single structural o...,ACA,It should be obvious why a single structural o...
2,It is not in those circumstances surprising th...,ACA,It is not in those circumstances surprising th...
3,Its significance lies in the seriousness of th...,ACA,Its significance lies in the seriousness of th...
4,This much is social democratic orthodoxy. It ...,ACA,This much is social democratic orthodoxy.
5,The relation between high activity of TNF and ...,ACA,The relation between high activity of TNF and ...
6,"The effects of humans can go far beyond this, ...",ACA,"The effects of humans can go far beyond this, ..."
7,"A combined transformation In a recent paper, H...",ACA,"A combined transformation In a recent paper, H..."
8,"However, while shoplifting occurred in the pas...",ACA,"However, while shoplifting occurred in the pas..."
9,Blocking will not usually increase the probabi...,ACA,Blocking will not usually increase the probabi...


In [ ]:
result_df.to_csv('/content/drive/MyDrive/Linguistic Data Science/data/4epoch-fic-output.csv', index=False, sep="|")